In [3]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

content = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

soup = BeautifulSoup(content, 'lxml')

data_table = soup.find('table', {'class': 'wikitable sortable'})
data_head = data_table.find_all('th')
db_head = []
for title in data_head:
  db_head.append(title.getText().strip())
db_head
data_body = data_table.find_all('td')
db_data = []
for item in data_body:
    db_data.append(item.getText().strip())


In [4]:
P, B, N = db_data[::3], db_data[1::3], db_data[2::3]
data = {db_head[0]: P, db_head[1]: B, db_head[2]: N}
df = pd.DataFrame(data)


In [5]:
delete_row = df[df["Borough"]=='Not assigned'].index
df = df.drop(delete_row)
df.head()


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [6]:
df1 = df.groupby('Postal code').agg({'Borough':'first',
                               'Neighborhood': ', '.join}).reset_index()


In [19]:
records = df1['Neighborhood'] == 'Not assigned'
result = filter(lambda x: x == True, records) 
count = len(list(result))
print('The number of Not assigned Neighborhoods  {}'.format(count))
df1.shape
df1.head(10)

The number of Not assigned Neighborhoods  0


,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


#Question 2

In [18]:
df3 = pd.read_csv ("http://cocl.us/Geospatial_data")
df.head(10)

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [21]:
import geocoder
from google.colab import files
uploaded = files.upload()


ModuleNotFoundError: No module named 'geocoder'

In [20]:
sudo pip3 install geocoder 


SyntaxError: invalid syntax (<ipython-input-20-36116e84c4bc>, line 1)

In [11]:
import io
df3 = pd.read_csv(io.BytesIO(uploaded['Geospatial_Coordinates.csv']))
df3.shape


NameError: name 'uploaded' is not defined

In [ ]:
df_2.rename(columns = {'Postal Code':'Postal code'}, inplace = True)
ds = df1.merge(df_2, on='Postal code')
ds.head(10)

In [12]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

ModuleNotFoundError: No module named 'geocoder'